# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [6]:
import $ivy.`org.scalaj::scalaj-http:2.4.2`

import $ivy.$                              

In [7]:
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [8]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

In [9]:
import $ivy.`org.json4s::json4s-jackson:3.4.0`
import org.json4s.JsonDSL._
import org.json4s._
import org.json4s.jackson.JsonMethods._
implicit val formats = DefaultFormats

import $ivy.$                                 

import org.json4s.JsonDSL._

import org.json4s._

import org.json4s.jackson.JsonMethods._

formats: DefaultFormats.type = org.json4s.DefaultFormats$@602878fb

In [10]:
import scala.collection.mutable.ListBuffer

import scala.collection.mutable.ListBuffer

### 1.1 Obtener el texto del RSS Feed y REDDIT Feed

Creamos una clase la cual toma un argumento que definira como se usara.
Consta de un metodo queryURL(url: String) ---> Seq[String] el cual recibe una url y devuelve una lista de texto, la cual tiene los datos de dicha url.

Para RSS:

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

Para REDDIT:

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse.Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML.Luego, se parsea con JSON para extraer los campos `title` y `selftext`.

In [132]:
val url1 = "https://www.reddit.com/r/Android/hot/.json?count=10"
val url2 = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

url1: String = "https://www.reddit.com/r/Android/hot/.json?count=10"
url2: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

In [133]:
abstract class URL_R{
    
    //Request HTTP
    def get_body(url: String): String = {
        try {
            Http(url).timeout(connTimeoutMs = 2000, readTimeoutMs = 5000).asString.body
        }
        catch{
            case e: Exception => ""   
        }
    }
    
    def parser(url: String): Seq[String]
}

defined class URL_R

In [134]:
class RSS_Parse extends URL_R {
    //Parse Rss
    def parser(url: String): Seq[String] = {
        val response = get_body(url)
        response match {
            case "" => Seq()
            case _ => val xml = XML.loadString(response)
                        // Extract text from title and description
                        (xml \\ "item").map { item => ((item \ "title").text + " " + (item \ "description").text) }
        }
        
    }
}

defined class RSS_Parse

In [135]:
class REDDIT_Parse extends URL_R {
    // Parse Reddit
    def parser(url: String): Seq[String] = {
        val response = get_body(url)
        response match {
            case "" => Seq()
            case _ => val result = (parse(response) \ "data" \ "children" \ "data")
                         .extract[List[Map[String, Any]]]
                    // Parsear JSON
                    val filterContent = result.flatten.filter{case (v , _) => v == "title" || v == "selftext" }.map(x => x._2.toString)
                    val pattern = "(https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]".r
                    filterContent.map(x => pattern.replaceAllIn(x,"")).toSeq
        }
    }
}

defined class REDDIT_Parse

In [136]:
val rss = new RSS_Parse()
val rssT = rss.parser(url2)

rss: RSS_Parse = ammonite.$sess.cmd133$Helper$RSS_Parse@5893e6ce
rssT: Seq[String] = List(
  "4 things we heard from Chicago Bears defensive coaches, including the competition at cornerback and Eddie Goldman\u2019s return from an opt-out year New Chicago Bears defensive coordinator Sean Desai and his revamped coaching staff will get their first chance to work with players on the field this weekend at rookie minicamp.",
  "After the Chicago Cubs won a team Gold Glove Award last season, a look at 4 reasons why their defense still is working to get back to elite form As the Chicago Cubs try to return to .500 this weekend in Detroit, it\u2019s worth delving into a few factors manager David Ross identified that he believes have affected the team\u2019s defensive performance.",
  "Prospect Henrik Borgstr\u00f6m already has personal connections to the Chicago Blackhawks \u2014 but the ones he\u2019s building with coaches matter most  The Chicago Blackhawks have high hopes for center Henrik Bo

In [137]:
val reddit = new REDDIT_Parse()
val redditT = reddit.parser(url1)

reddit: REDDIT_Parse = ammonite.$sess.cmd134$Helper$REDDIT_Parse@48123f5
redditT: Seq[String] = List(
  """**Credits to the team at /r/PickAnAndroidForMe for compiling this information:**

* Home - 

* Smartphones 101 - 

* Top Phones - 


***        
Note 1. Join us at /r/MoronicMondayAndroid, a sub serving as a repository for our retired weekly threads. Just pick any thread and Ctrl-F your way to wisdom! 

Note 2. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]()""",
  "What should I buy Thursday (May 13 2021) - Your weekly device inquiry thread!",
  """Hey /r/android, we had conducted a [feedback poll]() last year. We have decided to make a few revisions (mainly a preferential voting system) since some of the results were rather open-ended and did not present a clear solution. Participation on the last poll was also lower than anticipated since we only got about [930 responses]().

NOTES

* The poll was created via Google Forms and requires sign-in to 

## 2. Detectar las entidades nombradas

### 2.1 Crear la clase encargada del modelo,contar,y ordenar Entidades

Los metodos de esta clase son :

getNEs(textList: Seq[String]) ---> Seq[Seq[String]] el cual que recibe una lista de textos y aplica el modelo, para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

count(textL : Seq[Seq[String]]) ---> Map[String,Int] el cual recibe el resultado de aplicar el modelo, devuelve la entidad junto las cantidad de veces que aparece nombrada

sort(counts: Map[String,Int]) ---> List[(String, Int)] el cual ordena el resultado de contar las entidades

In [72]:
class NERModel() {
    // Variables Necesarias para crear modelo
    private val STOPWORDS = Seq (
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
        "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
        "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
        "their", "theirs", "themselves", "what", "which", "who", "whom",
        "this", "that", "these", "those", "am", "is", "are", "was", "were",
        "be", "been", "being", "have", "has", "had", "having", "do", "does",
        "did", "doing", "a", "an", "the", "and", "but", "if", "or",
        "because", "as", "until", "while", "of", "at", "by", "for", "with",
        "about", "against", "between", "into", "through", "during", "before",
        "after", "above", "below", "to", "from", "up", "down", "in", "out",
        "off", "over", "under", "again", "further", "then", "once", "here",
        "there", "when", "where", "why", "how", "all", "any", "both", "each",
        "few", "more", "most", "other", "some", "such", "no", "nor", "not",
        "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
        "will", "just", "don", "should", "now", "on",
        // Contractions without '
        "im", "ive", "id", "Youre", "youd", "youve",
        "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
        "theyre", "theyd", "theyve",
        "shouldnt", "couldnt", "musnt", "cant", "wont",
        // Common uppercase words
        "hi", "hello"
    )
    private val punctuationSymbols = ".,()!?;:'`´\n"
    private val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Aplicar el Modelo a los datos (simplemente es aplicar la funcion a la lista de textos)
    private def getNEsSingle(text: String): Seq[String] =
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq

    def getNEs(textList: Seq[String]): Seq[Seq[String]] = textList.map(getNEsSingle)
    
    // Contar las entidades
    def count(textL : Seq[Seq[String]]): Map[String,Int] = {
            textL.flatten.foldLeft(Map.empty[String, Int]) {
             (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    //Ordenar Entidades
    def sort(counts: Map[String,Int]): List[(String, Int)] = {
        counts.toList
          .sortBy(_._2)(Ordering[Int].reverse)
    }
}
       

defined class NERModel

In [14]:
val model = new NERModel

model: NERModel = ammonite.$sess.cmd12$Helper$NERModel@2e0a535e

### 2.2 Aplicar el "Modelo" a los datos

In [15]:
val result_rss = model.getNEs(rssT)
val result_reddit = model.getNEs(redditT)

result_rss: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Chicago",
    "White",
    "Sox",
    "Minnesota",
    "Twins",
    "Chicago",
    "White",
    "Sox",
    "Minnesota",
    "Twins",
    "Thursday",
    "Guaranteed",
    "Rate",
    "Field",
    "Lance",
    "Lynn"
  ),
  ArrayBuffer(
    "Photos",
    "Chicago",
    "White",
    "Sox",
    "Minnesota",
    "Twins",
    "Photos",
    "Chicago",
    "White",
    "Soxs",
    "Minnesota",
    "Twins",
    "May",
    "Guaranteed",
    "Rate",
    "Field",
    "Sox"
  ),
  ArrayBuffer("Gleyber", "Torres", "Yankee", "Yankees", "Torres"),
...
result_reddit: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Home",
    "Smartphones",
    "Top",
    "Phones",
    "Note",
    "Join",
    "Ctrl-F",
    "Note",
    "Join",
    "IRC",
    "Telegram"
  ),
  ArrayBuffer("Thursday", "May"),
  ArrayBuffer(
    "Hey",
    "Participation",
    "Google",
    "Forms",
    "Email",
    "Responses",
    "Well",
    "POLL]Edit"
  ),
  ArrayBuffer("Community"

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [16]:
val count_rss = model.count(result_rss)
val count_reddit = model.count(result_reddit)

count_rss: Map[String, Int] = Map(
  "Rate" -> 6,
  "Nets" -> 3,
  "Gleyber" -> 2,
  "Corey" -> 2,
  "Polisky" -> 6,
  "Operations" -> 1,
  "Parker" -> 2,
  "Crawford" -> 2,
  "Bieber" -> 2,
  "OC" -> 1,
  "Yankee" -> 1,
  "Sky" -> 2,
  "Washington" -> 1,
  "Robert" -> 1,
  "Dach\u2019s" -> 1,
  "Hockey" -> 1,
  "DeBrincat\u2019s" -> 1,
  "President" -> 1,
  "Alex" -> 1,
  "Abreu\u2019s" -> 1,
  "Shane" -> 2,
  "Finnish" -> 1,
  "COVID" -> 2,
  "Henrik" -> 2,
  "Toews\u2019" -> 2,
  "ERA" -> 1,
  "Bortles" -> 1,
  "Baldwin" -> 2,
  "Andrew" -> 2,
  "Stanley" -> 1,
  "MVP" -> 1,
  "Jim\u00e9nez" -> 1,
  "Kane\u2019s" -> 2,
  "Minnesota" -> 12,
  "Mike" -> 6,
  "Matt" -> 2,
  "WNBA\u2019s" -> 1,
  "Guaranteed" -> 6,
...
count_reddit: Map[String, Int] = Map(
  "Participation" -> 1,
  "Easier" -> 1,
  "Use" -> 1,
  "Well" -> 1,
  "Feedback" -> 1,
  "February" -> 1,
  "Central-" -> 1,
  "Zenfone" -> 8,
  "Xperia" -> 1,
  "IP68" -> 1,
  "Buds" -> 1,
  "PSA" -> 1,
  "PowerShell" -> 1,
  "Phon

In [17]:
val sort_rss = model.sort(count_rss)
val sort_reddit = model.sort(count_reddit)

sort_rss: List[(String, Int)] = List(
  ("Chicago", 46),
  ("White", 18),
  ("Sox", 18),
  ("Minnesota", 12),
  ("Twins", 12),
  ("Blackhawks", 11),
  ("Photos", 8),
  ("Cubs", 7),
  ("Bears", 7),
  ("Rate", 6),
  ("Polisky", 6),
  ("Mike", 6),
  ("Guaranteed", 6),
  ("Field", 6),
  ("NFL", 6),
  ("Bulls", 6),
  ("Patrick", 5),
  ("May", 5),
  ("Northwestern", 5),
  ("Tuesday", 4),
  ("COVID-19", 4),
  ("Fields", 4),
  ("Soxs", 4),
  ("Justin", 4),
  ("Cleveland", 4),
  ("Indians", 4),
  ("Nets", 3),
  ("Wednesday", 3),
  ("Brooklyn", 3),
  ("Alzolay", 3),
  ("Torres", 3),
  ("Milwaukee", 3),
  ("Friday", 3),
  ("Stan", 3),
  ("Zach", 3),
  ("Yankees", 3),
  ("Bowman", 3),
  ("Jonathan", 3),
...
sort_reddit: List[(String, Int)] = List(
  ("Flip", 10),
  ("Zenfone", 8),
  ("Google", 8),
  ("ZenFone", 6),
  ("Asus", 4),
  ("Android", 4),
  ("Galaxy", 3),
  ("Pixel", 3),
  ("ASUS", 3),
  ("Note", 2),
  ("Xiaomi", 2),
  ("US", 2),
  ("Hey", 2),
  ("Pro", 2),
  ("Europe", 2),
  ("Camera", 2

## 4. FeedService


In [83]:
class FeedService(){
    //Variable donde almacenar las suscripciones
    
    private val buffer = new ListBuffer[(String,URL_R)]()
    private val word = "%s".r
    //Guardar un registro de las URL, y opcionalmente sus parámetros, suscriptas.
    def suscribe(url: String, param: List[String], t: URL_R) : Unit= {
        param match {
            case Nil =>
                buffer.append((url,t))
            case param =>
                val url_t = param.map(x => word.replaceFirstIn(url,x)).map(x => (x,t))
                buffer ++= url_t
        }
    }
    
    //Obtener los feeds
    def get_feed(): Seq[Seq[String]] = {
        val toSeq = buffer.toSeq
        toSeq.map{ x => x._2.parser(x._1)}
    }
    
    //Compilar el resultado de cada una en una única lista
    def get_result(feed: Seq[Seq[String]]): Seq[String] = {
        feed.flatten
    }
    
}

defined class FeedService

In [84]:
val feed_service = new FeedService()
val url3 = "https://rss.nytimes.com/services/xml/rss/nyt/%s.xml"
val url4 = "https://www.chicagotribune.com/arcio/rss/category/%s/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val url5 = "https://www.reddit.com/r/%s/hot/.json?count=10"

feed_service: FeedService = ammonite.$sess.cmd82$Helper$FeedService@7b225695
url3: String = "https://rss.nytimes.com/services/xml/rss/nyt/%s.xml"
url4: String = "https://www.chicagotribune.com/arcio/rss/category/%s/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
url5: String = "https://www.reddit.com/r/%s/hot/.json?count=10"

In [85]:
feed_service.suscribe(url3, List[String]("Business","Technology"), new RSS_Parse())
feed_service.suscribe(url4, List[String]("sports","business"), new RSS_Parse())
feed_service.suscribe(url5, List[String]("Android"), new REDDIT_Parse())
feed_service.suscribe(url2, List[String](), new RSS_Parse())

In [86]:
val get_feed = feed_service.get_feed

get_feed: Seq[Seq[String]] = List(
  List(
    "Looking for Bipartisan Accord? Just Ask About Big Business. In surveys and political discourse, Republicans are increasingly critical of corporations, but not for the reasons Democrats have long held that view.Delta is among the corporations that have drawn fire from Republicans for taking stands against Georgia\u2019s new voting law after protests.",
    "He\u2019s a Dogecoin Millionaire. And He\u2019s Not Selling. Glauber Contessoto went looking for something that could change his fortunes overnight. He found it in a joke cryptocurrency.Glauber Contessoto believes deeply in Dogecoin.",
    "Retail Sales Were Flat in April Retail sales held steady in April after rising 10.7 percent the previous month, as Americans continued to spend government stimulus payments.Shoppers in New York this week. Retail sales were flat in April from the prior month.",
    "How to Navigate a Hot Housing Market Competition for homes in many cities is leading p

In [88]:
val get_result = feed_service.get_result(get_feed)

get_result: Seq[String] = List(
  "Looking for Bipartisan Accord? Just Ask About Big Business. In surveys and political discourse, Republicans are increasingly critical of corporations, but not for the reasons Democrats have long held that view.Delta is among the corporations that have drawn fire from Republicans for taking stands against Georgia\u2019s new voting law after protests.",
  "He\u2019s a Dogecoin Millionaire. And He\u2019s Not Selling. Glauber Contessoto went looking for something that could change his fortunes overnight. He found it in a joke cryptocurrency.Glauber Contessoto believes deeply in Dogecoin.",
  "Retail Sales Were Flat in April Retail sales held steady in April after rising 10.7 percent the previous month, as Americans continued to spend government stimulus payments.Shoppers in New York this week. Retail sales were flat in April from the prior month.",
  "How to Navigate a Hot Housing Market Competition for homes in many cities is leading potential buyers to 

In [89]:
val model_N = new NERModel

model_N: NERModel = ammonite.$sess.cmd71$Helper$NERModel@10b03898

In [90]:
val do_model = model_N.getNEs(get_result)
val do_count = model_N.count(do_model)
val do_sort = model_N.sort(do_count)

do_model: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Looking",
    "Bipartisan",
    "Accord",
    "Ask",
    "Big",
    "Business",
    "Republicans",
    "Democrats",
    "Republicans",
    "Georgia\u2019s"
  ),
  ArrayBuffer(
    "He\u2019s",
    "Dogecoin",
    "Millionaire",
    "He\u2019s",
    "Selling",
    "Glauber",
    "Contessoto",
    "Contessoto",
    "Dogecoin"
  ),
  ArrayBuffer(
    "Retail",
    "Sales",
    "Flat",
    "April",
    "Retail",
    "April",
    "Americans",
    "New",
    "York",
    "Retail",
    "April"
  ),
  ArrayBuffer("Navigate", "Hot", "Housing", "Market", "Competition"),
  ArrayBuffer(
...
do_count: Map[String, Int] = Map(
  "Thing" -> 2,
  "Talk" -> 1,
  "Much" -> 1,
  "Gcam" -> 1,
  "Dies" -> 1,
  "Rate" -> 10,
  "Amateur" -> 1,
  "Laws" -> 1,
  "Citing" -> 1,
  "Covid-19" -> 1,
  "Term" -> 1,
  "WHOA" -> 1,
  "Comes" -> 1,
  "Ahmed" -> 1,
  "Says" -> 1,
  "One" -> 1,
  "CEO" -> 3,
  "Metropolitan" -> 2,
  "PalestiniansThe" -> 2,
  "Protest"